In [1]:
! pip -q install transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, get_peft_model

# Carregar modelo base
device = "cuda"
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model = model.to(device)



tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
import json
f_train = open("/kaggle/input/dadostreinoval/formatted_data.json")
train_data = json.load(f_train)
f_train.close()
# print(len(train_data))


f_validate = open("/kaggle/input/dadostreinoval/valida.json")
validate_data = json.load(f_validate)
f_validate.close()
# print(len(validate_data))


In [4]:
train_data = [
    {
        "input": f"User: {pair[0]}",
        "label": f"Bot: {pair[1]}"
    }
    for pair in train_data
]


validate_data = [
    {
        "input": f"User: {pair[0]}",
        "label": f"Bot: {pair[1]}"
    }
    for pair in validate_data
]


In [ ]:
train_contexted = []

for i in range(len(train_data)):
    row = []
    row.append(train_data[i]["label"])  #resposta do bot
    row.append(train_data[i]["input"])  #pergunta do usuário
    train_contexted.append(row)


In [ ]:
validate_contexted = []

for i in range(len(validate_data)):
    row = []
    row.append(train_data[i]["label"])  #resposta do bot
    row.append(train_data[i]["input"])  #pergunta do usuário
    validate_contexted.append(row)  

In [7]:
columns = ['response', 'context'] 
columns = columns + ['context/'+str(i) for i in range(0)]
columns

['response', 'context']

In [8]:
import pandas as pd
len(train_contexted)
trn_df = pd.DataFrame.from_records(train_contexted, columns=columns)
trn_df.head(5)

,response,context
0,"Bot: If the belt is brown or black, consider w...",User: What outfit can I wear to highlight the ...
1,"Bot: For pants, brownish or gray tones work we...",User: What colors should I use in the outfit w...
2,Bot: It's one of the possibilities. This is a ...,"User: For this style with light fabric pants, ..."
3,"Bot: Yes, you can! In this case, consider wear...","User: Can I use a lighter style of pants, simi..."
4,Bot: Consider wearing straight-leg jeans with ...,"User: For a look highlighting the belt, outsid..."


In [9]:
len(validate_contexted)
val_df = pd.DataFrame.from_records(validate_contexted, columns=columns)
val_df.head(5)

,response,context
0,"Bot: If the belt is brown or black, consider w...",User: What outfit can I wear to highlight the ...
1,"Bot: For pants, brownish or gray tones work we...",User: What colors should I use in the outfit w...
2,Bot: It's one of the possibilities. This is a ...,"User: For this style with light fabric pants, ..."
3,"Bot: Yes, you can! In this case, consider wear...","User: Can I use a lighter style of pants, simi..."
4,Bot: Consider wearing straight-leg jeans with ...,"User: For a look highlighting the belt, outsid..."


In [ ]:
from transformers import AutoTokenizer

#carregar o tokenizer

def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv



tokenized_train=[]
tokenized_teste=[]


for _, row in trn_df.iterrows():
    conv = construct_conv(row, tokenizer)
    tokenized_train.append(conv)



for _, row in val_df.iterrows():
    conv = construct_conv(row, tokenizer)
    tokenized_teste.append(conv)


In [11]:
from transformers import Trainer, TrainingArguments
from transformers import AdamW
import torch



In [ ]:
def compute_perplexity(p):
    return math.exp(p)  #exponenciação da perda para calcular a perplexidade

#função de métricas personalizadas
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    #a perda é calculada com logits e labels
    loss = torch.nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1), reduction='mean')
    #calcular a perplexidade
    perplexity = compute_perplexity(loss.item())
    return {"loss": loss.item(), "perplexity": perplexity}

In [ ]:
from transformers import Trainer, TrainingArguments,DataCollatorForLanguageModeling
from transformers import AdamW


if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)


training_args = TrainingArguments(
    output_dir="./moda_modelo",
    num_train_epochs=8,#3
    logging_strategy="steps",
    logging_steps=1,
    # learning_rate = 5e-5, #NEW!
    fp16=True,
    weight_decay=0.01,  #adiciona decaimento de peso
    adam_epsilon=1e-8,  #valor para o epsilon do Adam
    max_grad_norm=1.0,  #radientes
    per_device_train_batch_size=8,#8
    logging_dir="./logs",
    report_to="none",  
    evaluation_strategy="epoch",  
    eval_steps=None,  
)

optimizer = AdamW(model.parameters(), lr=training_args.learning_rate, eps=training_args.adam_epsilon, weight_decay=training_args.weight_decay)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,  
    eval_dataset=tokenized_teste,    
    optimizers=(optimizer, None),  
    data_collator=data_collator,
    # compute_metrics=compute_metrics  

)

result= trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.610800,1.847499
2,1.442800,1.401899
3,1.275200,1.177736
4,1.062300,1.005713
5,1.096000,0.885593
6,0.903800,0.789580
7,0.894200,0.738745
8,0.874500,0.719216


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [14]:
trainer.save_model("./moda_modelo/final")
tokenizer.save_pretrained("./moda_modelo/final")


('./moda_modelo/final/tokenizer_config.json',
 './moda_modelo/final/special_tokens_map.json',
 './moda_modelo/final/vocab.json',
 './moda_modelo/final/merges.txt',
 './moda_modelo/final/added_tokens.json',
 './moda_modelo/final/tokenizer.json')

In [15]:
result.metrics

{'train_runtime': 521.1601,
 'train_samples_per_second': 19.648,
 'train_steps_per_second': 1.228,
 'total_flos': 1154229853814784.0,
 'train_loss': 1.2129270805045962,
 'epoch': 8.0}

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "/kaggle/working/moda_modelo/final"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)


In [ ]:

def generate_text(prompt, model, tokenizer, max_length=256):

    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

    #nputs para o mesmo dispositivo que o modelo
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    #gerar texto com controles para evitar repetição
    response_ids = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,           
        no_repeat_ngram_size=1,          
        temperature=0.7,               
        top_k=50,                        
        top_p=0.9,                       
        do_sample=True,                  
    )

    #decodificar a resposta gerada
    response = tokenizer.decode(response_ids[0], skip_special_tokens=True)
    return response

#gerar uma resposta
prompt = "i am a woman. Can you reccomend a outfit that make use of a hoodie, what is the best choice of cloths to make a outstanding outfit?"
generated_text = generate_text(prompt, model, tokenizer)
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


i am a woman. Can you reccomend a outfit that make use of a hoodie, what is the best choice of cloths to make a outstanding outfit? Hoodies are versatile and easy for women's clothesTo wear: Women can opt For oversized or fitted tops with jeans In winter! Men love long coats in warmer weather


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM


def generate_response(prompt):
    #codificar o prompt
    bot_input_ids = tokenizer.encode(prompt + tokenizer.eos_token, return_tensors='pt')

    chat_history_ids = model.generate(
        bot_input_ids, max_length=256,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature=0.7
    )

    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response

prompt = "i am a woman. Can you reccomend a outfit that make use of a hoodie, what is the best choice of cloths to make a outstanding outfit?"
response = generate_response(prompt)
print("Response: ", response)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Response:  A hoody and denim jacket are both stylish and functional. A denim skirt or a long coat will keep you comfortable and stylish. Pair with sneakers for a sporty look. For a casual style, pair with jeans and boots.
